In [ ]:
# torch with cuda
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from ultralytics import YOLO
import torch
import os

# Load a model
# model = YOLO("best.onnx")
yolo_model = YOLO("yolo11n.pt")

path = os.path.join(os.path.abspath("."),"License Plate Recognition.v6i.yolov11.zip")

# # Train the model
# train_results = model.train(
#     data=path,  # path to dataset YAML
#     epochs=100,  # number of training epochs
#     # workers=100,
#     # patience=20,
#     imgsz=640,  # training image size
#     device="cuda" if torch.cuda.is_available() else "cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
# )

# # Evaluate model performance on the validation set
# metrics = model.val()

# # Export the model to ONNX format
# path = model.export(format="onnx")  # return path to exported model

In [8]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from ultralytics import YOLO
from ultralytics.engine.results import Boxes, Results
import os
import cv2
from matplotlib import pyplot as plt
from safe_video import NumberPlateRecognition

yolo_model_path = os.path.join(os.path.abspath("."), "models", "yolo11n.pt")
yolo_model = YOLO(yolo_model_path, task='detect')

rec = NumberPlateRecognition()
# image = cv2.imread("test2.png")

# image = image[:, :, ::-1]
# box, conf, _, _ = rec.chained_detection(image, "car").tuple()
# result = rec.blur_image(image, box)
# plt.imshow(result)
# plt.show()

In [ ]:
display_width = 1200
display_height = 800

cap = cv2.VideoCapture(r"Video here")

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = yolo_model.track(frame, persist=True) # yolo_model(frame, imgsz=320 ,verbose = False)
        # display(results[0].boxes)
        #annotated_frames = results[0].plot() if results[0] is not None else frame
        
        #frames = results[0].plot() if results[0] is not None else Boxes()
        boxes = results[0].boxes if results[0] is not None else Boxes()
        if boxes:
            for box in boxes.data:
                x1, y1, x2, y2 = map(int, box[:4])  
                conf = box[5]
                cls = int(box[6])
                cropeed_image = frame[y1:y2, x1:x2]
                
                if conf > 0.5:  
                    box, conf, _, _ = rec.chained_detection(cropeed_image, "car").tuple()
                    display(conf.shape)
                    result = rec.blur_image(cropeed_image, box)
                    frame[y1:y2, x1:x2] = result
        resized_frame = cv2.resize(frame, (display_width, display_height))
        
        cv2.imshow("Tracking", resized_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()
                
                
    


0: 384x640 13 cars, 2 trucks, 21.0ms
Speed: 2.0ms preprocess, 21.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 car, 15.0ms
Speed: 1.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 License_Plate, 15.0ms
Speed: 2.5ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


(1,)


0: 480x640 1 car, 13.5ms
Speed: 2.0ms preprocess, 13.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


(0,)


0: 416x640 1 car, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)


...

(0,)


0: 384x640 2 donuts, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


(0,)


0: 384x640 1 hot dog, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


(0,)